# Majority vote labeling

In [1]:
from label_legends.preprocess import load_data, transform, majority
import polars as pl

/Users/retipeter/Documents/TU Wien/NLP/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Function that is basically the majority in the preprocessing.py

In [2]:
df1 = load_data().collect()
df2 = transform(df1)
df2 = df2.with_columns(df1.select("rewire_id"))
df2 = df2.with_columns(
    pl.col("rewire_id").str.extract(r"(\d+)$", 1).cast(pl.Int64).alias("rewire_id_number")
)
set_label = df2.group_by("rewire_id_number").agg([
    pl.col("label").sum().alias("label_sum"),
    pl.col("text").first().alias("text"),
    pl.col("tokens").first().alias("tokens"),
    pl.col("token_ids").first().alias("token_ids")
    ]
)
set_label = set_label.with_columns(
    pl.when(pl.col("label_sum") >= 2).then(1).otherwise(0).alias("label")  # Conditional update for 'label'
)
set_label

/Users/retipeter/Documents/TU Wien/NLP/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


rewire_id_number,label_sum,text,tokens,token_ids,label
i64,i64,str,list[str],list[i64],i32
16804,0,"""This boomer seems very angry a…","[""boomer"", ""angry"", … "".""]","[394, 201, … 26]",0
17334,2,"""get a thick book and cut a hol…","[""book"", ""cut"", … "".""]","[393, 726, … 26]",1
10120,1,"""Exactly, it was a non harmful …","[""exactly"", "","", … "".""]","[975, 22, … 26]",0
14711,3,"""The vast majority of my friend…","[""vast"", ""majority"", … ""bang""]","[2849, 1647, … 307]",1
155,1,"""sure can be... look at the For…","[""sure"", ""..."", … "".""]","[2621, 28, … 26]",0
…,…,…,…,…,…
5886,2,"""do a stone ole stunna on may t…","[""stone"", ""ole"", … ""fuka""]","[2567, 1891, … 3000]",1
6928,0,"""Got any advice on getting rid …","[""advice"", ""rid"", … ""haha""]","[154, 2271, … 1224]",0
12710,0,""". wants a federal court to blo…","[""."", ""want"", … "".""]","[26, 2881, … 26]",0


In [3]:
majority()

rewire_id_number,label_sum,text,tokens,token_ids,label
i64,i64,str,list[str],list[i64],i32
16405,2,"""If you were born male you lost…","[""bear"", ""male"", … "".""]","[325, 1650, … 26]",1
8816,0,"""Good, let's start with her.""","[""good"", "","", … "".""]","[1181, 22, … 26]",0
14446,3,"""Trump's America where police t…","[""trump"", ""'s"", … ""😂""]","[2778, 15, … 2994]",1
7229,0,"""Nothing wrong with being a str…","[""wrong"", ""stripper"", … "".""]","[2959, 3000, … 26]",0
16140,0,"""#NRA spokeswoman #DanaLoesch s…","[""#"", ""nra"", … "":""]","[7, 3000, … 86]",0
…,…,…,…,…,…
9685,0,"""Stands by her man! ""","[""stand"", ""man"", ""!""]","[2544, 1652, 0]",0
15961,2,"""Well, women do have high AF se…","["","", ""woman"", … ""case""]","[22, 2939, … 493]",1
14660,0,"""You're the one saying your pol…","[""say"", ""political"", … ""?""]","[2331, 2025, … 98]",0
